# Лабораторная 2. Формирование отчётов в Apache Spark

## Задание

Сформировать отчёт с информацией о 10 наиболее популярных языках программирования по итогам года за период с 2010 по 2020 годы. Отчёт будет отражать динамику изменения популярности языков программирования и представлять собой набор таблиц "топ-10" для каждого года.

Получившийся отчёт сохранить в формате Apache Parquet.

Для выполнения задания вы можете использовать любую комбинацию Spark API: RDD API, Dataset API, SQL API.


In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, rank
from pyspark.sql.window import Window

In [ ]:
import xml.etree.ElementTree as ET

# Путь к XML-файлу
xml_path = 'posts_sample.xml'

# Парсинг XML и получение корневого элемента
tree = ET.parse(xml_path)
root_elem = tree.getroot()

# Формируем список кортежей (creation_date, tag) с использованием list comprehension и оператора присваивания
records = [
    (creation_date, tag)
    for row in root_elem.findall('row')
    if (creation_date := row.attrib.get('CreationDate')) and (tags_str := row.attrib.get('Tags'))
    for tag in tags_str.replace('<', '').replace('>', ' ').strip().split()
]

for record in records[:10]:
    print(record)


('2008-07-31T21:42:52.667', 'c#')
('2008-07-31T21:42:52.667', 'floating-point')
('2008-07-31T21:42:52.667', 'type-conversion')
('2008-07-31T21:42:52.667', 'double')
('2008-07-31T21:42:52.667', 'decimal')
('2008-07-31T22:08:08.620', 'html')
('2008-07-31T22:08:08.620', 'css')
('2008-07-31T22:08:08.620', 'internet-explorer-7')
('2008-07-31T23:40:59.743', 'c#')
('2008-07-31T23:40:59.743', '.net')


: 